In [ ]:
import pandas as pd
import glob
from igc_lib.igc_lib import Flight
from pdflight import flights_to_dataframes, shift_datetime
from joblib import Parallel, delayed
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
flights = Parallel(n_jobs=-1)(
    delayed(Flight.create_from_file)(i) for i in glob.glob("./*.igc")
)

In [ ]:
md, fl, th = flights_to_dataframes(
    (flight for flight in flights if flight.valid and flight.task_finish_time)
)
%store md fl th

In [ ]:
%store -r md fl th

In [ ]:
md.competition.unique()

In [ ]:
tt = th.loc[fl.loc[fl.index, "task"]]

In [ ]:
df = pd.DataFrame(index=tt.index)
for i in tt.index:
    df.at[i, "v"] = (
        fl.loc[shift_datetime(i, tt.loc[i])].alt - fl.loc[i].alt
    ) / (tt.loc[i].total_seconds())

In [ ]:
cc = md.join(df.v.groupby(["code", "id", "date"]).mean(), how="inner")
cc

In [ ]:
sns.regplot(data=cc, x='v', y='points', line_kws={"color": "red"})
plt.show()

In [ ]:
res = stats.linregress(cc.v, cc.points)
res

In [ ]:
# Die Formel y = m*x + b auf die ganze Spalte anwenden
cc['points_pred'] = res.slope * cc.v + res.intercept

# Optional: Das Residuum (Abweichung: Wie viel besser/schlechter war der Flug als der Schnitt?)
cc['residual'] = cc.points - cc.points_pred

In [ ]:
cc.groupby("pilot").sum().sort_values("residual")

In [ ]:
sns.regplot(data=cc.loc[:,["residual", "points", "pilot"]].groupby("pilot").mean(), x='residual', y='points', line_kws={"color": "red"})

In [ ]:
stats.linregress(cc.groupby("pilot").residual.sum(), cc.groupby("pilot").points.sum())